# Gemma models testing

## Preprocessing

In [ ]:
from pdf_parser import *
import os

pdf_folder_path = "../data/PDF"
# pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith('.pdf')]
pdf_files = ["24EM03456.pdf"]
image_save_folder_path = "../data/images"

pdf_text_image = {}
for pdf_file in pdf_files:
    #could use langchain here to extract text from pdf and use Document object
    pdf_text_image[os.path.splitext(pdf_file)[0]] = {"text":extract_with_pdfplumber(os.path.join(pdf_folder_path,pdf_file))}
    pdf_images = extract_pdf2image(f"{pdf_folder_path}/{pdf_file}")
    pdf_text_image[os.path.splitext(pdf_file)[0]]["image"] = pdf_images

print(pdf_text_image.keys())
print(pdf_text_image)


2025-05-23 14:35:15,207 CropBox missing from /Page, defaulting to MediaBox
2025-05-23 14:35:15,209 CropBox missing from /Page, defaulting to MediaBox
2025-05-23 14:35:15,210 CropBox missing from /Page, defaulting to MediaBox
2025-05-23 14:35:15,612 CropBox missing from /Page, defaulting to MediaBox
2025-05-23 14:35:15,613 CropBox missing from /Page, defaulting to MediaBox
2025-05-23 14:35:15,614 CropBox missing from /Page, defaulting to MediaBox
2025-05-23 14:35:15,615 Text extracted from the PDF file


dict_keys(['24EM03456'])
{'24EM03456': {'text': "COPIE INTERNE 29/10/2024\nDr ROGER Thierry\nCHIREC DELTA\nRadiodiagnostic\nCentre d’Anatomie BOULEVARD DU TRIOMPHE 1\nPathologique H.U.B. 1160 AUDERGHEM\nRue Meylemeersch 90 - 1070 Anderlecht\nMijlemeerschstraat 90 – 1070 Anderlecht\nPATIENT :\nDirectrice de Service\nPr Myriam Remmelink\nID :\nRéf. Externe : 24CU052383 EXAMEN : 24EM03456\nEquipe Médicale\nDr Nicolas de Saint Aubain\nPr Nicky D’Haene Prélevé le 30/07/2024 à 30/07/2024 11:30 Prescripteur : Dr ROGER Thierry\nDr Maria Gomez Galdon Reçu le 08/08/2024\nDr Chirine Khaled\nPr Denis Larsimont\nDr Laetitia Lebrun\nDr Calliope Maris\nPr Jean-Christophe Noël\nPr Isabelle Salmon\nDr Anne-Laure Trépant\nPr Laurine Verset\nRECHERCHE PAR « NEXT GENERATION SEQUENCING » DE\nConsultant (e) s\nDr Sarah Bouri MUTATIONS DANS 25 GENES IMPLIQUES DANS LES CANCERS\nDr Xavier Catteau PULMONAIRES, LES GIST ET MELANOMES\nDr Roland de Wind\nDr Valérie Segers (Colon and Lung Panel + Oncomine Solid Tum

In [2]:
with open("../prompt/system_prompt_metadata.md", "r") as f:
    system_prompt_meta = f.read()
print(system_prompt_meta)

with open("../prompt/system_prompt_mutation.md", "r") as f:
    system_prompt_mut = f.read()
print(system_prompt_mut)

# Extraction de données depuis un rapport de biopsie

Tu es un assistant médical intelligent spécialisé en oncologie.  
Ta mission est d’extraire de manière fiable et structurée les **informations médicales clés** à partir de **rapports de biopsie** provenant de services d'oncologie hospitaliers.

Tu recevras un **document textuel**, ou le **texte OCR extrait d'une image de rapport**, contenant les informations nécessaires.  
Ta sortie doit être un **dictionnaire JSON structuré** et exploitable.

---

## Objectifs

1. Lire et comprendre le contenu d’un rapport de biopsie.
2. Identifier et extraire les informations cliniques spécifiques listées ci-dessous.
3. Structurer les informations extraites sous forme de dictionnaire conforme.

---

## Informations à extraire

Pour chaque rapport, extrais les données suivantes :

| Clé du dictionnaire       | Description                                                                 |
|---------------------------|---------------------------------

## Model definition


In [ ]:
import torch
from huggingface_hub import login

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

with open("../login.txt", "r") as f:
    token = f.read()
login(token) #token from huggingface.co necessary to use gemma3
print("login done")

### Gemma 3

In [ ]:
from transformers import pipeline

model_name = "google/gemma-3-4b-it"

pipe = pipeline(
    "text-generation", #"image-text-to-text"
    model=model_name,
    torch_dtype=torch.bfloat16,
    device=device, #uses "cpu" here because the "cuda" requires 3 Go more of VRAM (5GO total)
)
print("pipeline initialized")

messages_meta = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt_meta}]
    },
    {
        "role": "user",
        "content": (
            [{"type": "text", "text": pdf_text_image["24EM03456"]["text"]}]
            #+ [{"type": "image", "image": img} for img in pdf_text_image["24EM03352"]["image"]]
        )
    }
]

messages_mut = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt_mut}]
    },
    {
        "role": "user",
        "content": (
            [{"type": "text", "text": pdf_text_image["24EM03456"]["text"]}]
            #+ [{"type": "image", "image": img} for img in pdf_text_image["24EM03352"]["image"]]
        )
    }
]

output_gemma_4B = pipe(messages_meta, max_new_tokens=250) #temperature=0
print(output_gemma_4B[0]["generated_text"][-1]["content"])
output_gemma_4B2 = pipe(messages_mut, max_new_tokens=250) #temperature=0
print(output_gemma_4B2[0]["generated_text"][-1]["content"])

login done


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.64it/s]
Device set to use cpu


pipeline initialized
```json
{
  "Examen": "24EM03456",
  "N° du prélèvement": "24CU052383",
  "Panel": "CLP",
  "Origine du prélèvement": "Curepath",
  "Type de prélèvement": "Adénocarcinome TTF1+",
  "Qualité du séquencage": "Optimale",
  "% cellules": "10"
}
```



### Medgemma 3
When running on cpu, 38min inference time for 1 request

In [ ]:
from transformers import pipeline
import torch

model_name = "google/medgemma-4b-it"

pipe = pipeline(
    "text-generation", #"image-text-to-text"
    model=model_name,
    torch_dtype=torch.bfloat16,
    device=device, #uses "cpu" here because the "cuda" requires 3 Go more of VRAM (5GO total)
)
print("pipeline initialized")

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt_meta}]
    },
    {
        "role": "user",
        "content": (
            [{"type": "text", "text": pdf_text_image["24EM03456"]["text"]}]
            #+ [{"type": "image", "image": img} for img in pdf_text_image["24EM03352"]["image"]]
        )
    }
]

output_medgemma = pipe(messages, max_new_tokens=250) #temperature=0
print(output_medgemma[0]["generated_text"][-1]["content"])

login done


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.69it/s]
Device set to use cpu


pipeline initialized
```json
{
  "Examen": "24EM03456",
  "N° du prélèvement": "24CU052383 pneu",
  "Panel": "CLP",
  "Origine du prélèvement": "Curepath",
  "Type de prélèvement": "Adénocarcinome TTF1+",
  "Qualité du séquencage": "Optimale",
  "% cellules": "2"
}
```


### Gemma3 1B
6 minutes for inference on 1 request

In [ ]:
from transformers import pipeline
import torch

model_name = "google/gemma-3-1b-it" # google gemma small
# model_name = "google/gemma-3-4b-it-qat-q4_0-gguf" #quantization form of the model

pipe = pipeline(
    "text-generation",
    model=model_name,
    torch_dtype=torch.bfloat16,
    device=device, #uses "cpu" here because the "cuda" requires 3 Go more of VRAM (5GO total)
)
print("pipeline initialized")

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt_meta}]
    },
    {
        "role": "user",
        "content": (
            [{"type": "text", "text": pdf_text_image["24EM03456"]["text"]}]
        )
    }
]

# the max_new_tokens is related to the time spent to generate the answer
output_gemma_1B = pipe(messages, max_new_tokens=250) #can add temperature=0
print(output_gemma_1B[0]["generated_text"][-1]["content"])

login done


Device set to use cpu


pipeline initialized
```json
{
  "Examen": "24EM03456",
  "N° du prélèvement": "24MH9721 BN",
  "Panel": "CLP",
  "Origine du prélèvement": "Centre Hospitalier de Mouscron",
  "Type de prélèvement": "Adénocarcinome lieberkühnien",
  "Qualité du séquencage": "Optimale",
  "% cellules": 50
}
```


### Gemma3 4B quantized
Quantized models can't be used with hugging face library.
It need llama.cpp or ollama to run.

In [ ]:
model_name = "google/gemma-3-4b-it-qat-q4_0-gguf"

login done


ValueError: Could not load model google/gemma-3-4b-it-qat-q4_0-gguf with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/home/alexa/.local/lib/python3.10/site-packages/transformers/pipelines/base.py", line 291, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "/home/alexa/.local/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py", line 571, in from_pretrained
    return model_class.from_pretrained(
  File "/home/alexa/.local/lib/python3.10/site-packages/transformers/modeling_utils.py", line 279, in _wrapper
    return func(*args, **kwargs)
  File "/home/alexa/.local/lib/python3.10/site-packages/transformers/modeling_utils.py", line 4260, in from_pretrained
    check

Show results from experiments

In [ ]:
#save the output in json format
import json
import pandas as pd

answer_meta = output[0]["generated_text"][-1]["content"]
cleaned_meta = answer_meta.replace("```json", "").replace("```", "").strip() #clean the answer
data = json.loads(cleaned_meta)
print(data)

answer_mut = output2[0]["generated_text"][-1]["content"]
cleaned_mut = answer_mut.replace("```json", "").replace("```", "").strip() #clean the answer
data2 = json.loads(cleaned_mut)
print(data2)

df = pd.DataFrame([data])
df.to_excel("../out/metadata_gemma3_4B.xlsx", index=False)
df2 = pd.DataFrame(data2)
df2.to_excel("../out/mutation_gemma3_4B.xlsx", index=False)

NameError: name 'output' is not defined